In [ ]:
import pandas as pd
import random
import numpy as np

In [ ]:
random.seed(42)
np.random.seed(42)

In [ ]:
data = pd.read_csv("all_data.csv")
data.head()

In [ ]:
if 'experience_maps_to_save' not in globals():
    experience_maps_to_save = {}
if 'grouped_imputation_maps_to_save' not in globals():
    grouped_imputation_maps_to_save = {}

In [ ]:
data.columns

In [ ]:
data['worldwide']

In [ ]:
nan_data = (data.isnull().mean() * 100).reset_index()
nan_data.columns = ["column_name", "percentage"]
nan_data.sort_values("percentage", ascending=False, inplace=True)
nan_data.head(24)

In [ ]:
data = data.dropna(subset=['worldwide'])

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
test_data.to_csv("test.csv", index=False)

In [ ]:
train_movie_title = train_data['movie_title']
columns_to_drop = ['movie_id', 'movie_title', 'link']
#columns_to_drop = ['movie_id', 'link']
train_data = train_data.drop(columns=columns_to_drop)
train_data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style="whitegrid")


plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")

# 1. Heatmap корреляций числовых признаков
# Выбираем только числовые колонки
numeric_cols = train_data.select_dtypes(include=[np.number]).columns

# Считаем корреляции и сортируем по целевой переменной
corr_matrix = train_data[numeric_cols].corr()
target_corr = corr_matrix['worldwide'].sort_values(ascending=False)

# Визуализируем топ-20 признаков по корреляции
plt.figure(figsize=(12, 10))
top_features = target_corr.index[1:21]  # исключаем сам worldwide
sns.heatmap(train_data[top_features].corr(), 
            annot=True, fmt=".2f", 
            cmap='coolwarm', 
            center=0,
            vmin=-1, vmax=1,
            linewidths=0.5)
plt.title('Тепловая карта корреляций (топ-20 признаков)', fontsize=16)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Распределение целевой переменной
plt.figure(figsize=(12, 6))

# Гистограмма
plt.subplot(1, 2, 1)
sns.histplot(train_data['worldwide'], kde=True, bins=50)
plt.title(f'Распределение worldwide\nSkewness: {train_data["worldwide"].skew():.2f}')
plt.xlabel('Цена ($)')
plt.ylabel('Частота')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=train_data['worldwide'])
plt.title('Boxplot worldwide')
plt.ylabel('Цена ($)')

plt.suptitle('Анализ целевой переменной', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
cols = ["budget", "domestic", "international", "worldwide", "run_time"]

for c in cols:
    plt.figure(figsize=(8, 4))
    sns.histplot(train_data[c].dropna(), kde=True, stat="density", bins=30)
    plt.title(f"Распределение и KDE для {c}")
    plt.xlabel(c)
    plt.ylabel("Плотность")
    plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=train_data[["domestic", "international", "worldwide"]])
plt.title("Boxplot: domestic vs international vs worldwide")
plt.ylabel("Сборы")
plt.show()


In [ ]:
# Добавляем колонку с декадой
train_data["decade"] = (train_data["movie_year"] // 10) * 10

plt.figure(figsize=(12, 6))
sns.boxplot(x="decade", y="budget", data=train_data)
plt.title("Бюджет по десятилетиям выпуска")
plt.xlabel("Десятилетие")
plt.ylabel("Бюджет")
plt.xticks(rotation=45)
plt.show()
train_data = train_data.drop("decade", axis=1)


In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(
    np.log1p(train_data["budget"]),
    np.log1p(train_data["worldwide"]),
    alpha=0.5
)
plt.title("log(budget) vs log(worldwide)")
plt.xlabel("log(Бюджет +1)")
plt.ylabel("log(Сборы worldwide +1)")
plt.show()


In [ ]:
plt.figure(figsize=(6, 6))
sns.scatterplot(x="run_time", y="worldwide", data=train_data, alpha=0.6)
plt.title("run_time vs worldwide")
plt.xlabel("run_time (мин)")
plt.ylabel("worldwide")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.regplot(x="movie_year", y="worldwide", data=train_data, scatter_kws={"s":30, "alpha":0.5})
plt.title("Год выпуска vs worldwide (линейный тренд)")
plt.xlabel("Год выпуска")
plt.ylabel("worldwide")
plt.show()

In [ ]:
for g in ["genre_1","genre_2","genre_3","genre_4"]:
    stats = (
        train_data
        .dropna(subset=[g])
        .groupby(g)["worldwide"]
        .agg(["count","median"])
        .reset_index()
        .sort_values("count", ascending=False)
    )
    plt.figure(figsize=(8, 5))
    sns.barplot(x=g, y="median", data=stats, palette="viridis")
    plt.title(f"Медианные сборы worldwide по {g} (частота показана числом над столбцами)")
    plt.xlabel(g)
    plt.ylabel("Медианные сборы")
    for idx, row in stats.iterrows():
        plt.text(idx, row["median"] + 0.02*row["median"], int(row["count"]), 
                 ha="center", va="bottom", fontsize=9)
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# ==============================================================================
# >>> ОБНОВЛЕННЫЙ БЛОК: Создание признаков опыта и СОХРАНЕНИЕ КАРТ <<<
# ==============================================================================

# Инициализируем словари для карт, которые мы хотим сохранить
# Это нужно сделать один раз где-то в начале вашего скрипта предобработки,
# если вы еще этого не сделали. Если уже сделали, эту строку можно пропустить.
# Для примера, я добавлю инициализацию здесь, но лучше вынести ее выше.
if 'experience_maps_to_save' not in globals(): # Проверяем, существует ли переменная
    experience_maps_to_save = {}
    print("Инициализирован 'experience_maps_to_save'")

personnel_cols_for_experience = ["director", "writer", "producer", "composer", "cinematographer"]
print("\nСоздание признаков опыта для съемочной группы и сохранение карт...")
for col in personnel_cols_for_experience:
    if col in train_data.columns:
        counts = train_data[col].value_counts()
        # СОХРАНЯЕМ КАРТУ (словарь: имя -> количество)
        experience_maps_to_save[f"{col}_experience_map"] = counts.to_dict()
        
        train_data[f"{col}_experience"] = train_data[col].map(counts)
        # Заполняем пропуски нулями, если после map остались NaN 
        # (например, для значений, которых не было в train_data[col] при подсчете counts,
        # или если train_data[col] имел NaN)
        train_data[f"{col}_experience"].fillna(0, inplace=True)
        print(f"  Создан признак {col}_experience и сохранена карта.")
    else:
        print(f"  Предупреждение: столбец {col} не найден в train_data для создания опыта.")

# Опыт главных актёров и суммарная "звёздность"
actor_experience_cols_generated = [] # Будем хранить имена созданных колонок опыта
print("\nСоздание признаков опыта для актеров и сохранение карт...")
for i in range(1, 5): # main_actor_1, main_actor_2, main_actor_3, main_actor_4
    col_actor = f"main_actor_{i}"
    if col_actor in train_data.columns:
        counts_actor = train_data[col_actor].value_counts()
        # СОХРАНЯЕМ КАРТУ
        experience_maps_to_save[f"{col_actor}_experience_map"] = counts_actor.to_dict()
        
        experience_col_name = f"{col_actor}_experience"
        train_data[experience_col_name] = train_data[col_actor].map(counts_actor)
        train_data[experience_col_name].fillna(0, inplace=True)
        actor_experience_cols_generated.append(experience_col_name)
        print(f"  Создан признак {experience_col_name} и сохранена карта.")
    else:
        print(f"  Предупреждение: столбец {col_actor} не найден для создания опыта.")

# Суммарная "звёздность"
if actor_experience_cols_generated: # Только если были созданы колонки опыта актеров
    train_data["cast_popularity"] = train_data[actor_experience_cols_generated].sum(axis=1)
    print("  Создан признак cast_popularity.")
else:
    train_data["cast_popularity"] = 0 
    print("  Признак cast_popularity установлен в 0 (колонки опыта актеров не созданы).")

print("\nСловарь experience_maps_to_save (пример):")
# Выведем первые несколько ключей для проверки
for k in list(experience_maps_to_save.keys())[:2]:
    print(f"  {k}: {list(experience_maps_to_save[k].items())[:2]}...") # Первые 2 элемента карты

# ==============================================================================
# >>> КОНЕЦ ОБНОВЛЕННОГО БЛОКА <<<
# ==============================================================================

In [ ]:
import re

def convert_runtime_to_minutes(value):
    match = re.match(r'(?:(\d+)\s*hr)?\s*(?:(\d+)\s*min)?', str(value))
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        return hours * 60 + minutes
    return None  # если формат не распознан

train_data['run_time'] = train_data['run_time'].apply(convert_runtime_to_minutes)


In [ ]:
from sklearn.impute import SimpleImputer

numerical_cols_for_imputation_training = [
    col for col in train_data.select_dtypes(include=[np.number]).columns 
    if col not in ['worldwide', 'domestic', 'international'] # Исключаем!
]
if numerical_cols_for_imputation_training: # Только если есть что импьютировать
    num_imputer = SimpleImputer(strategy='median')
    train_data[numerical_cols_for_imputation_training] = \
        num_imputer.fit_transform(train_data[numerical_cols_for_imputation_training])
    print(f"Numerical Imputer обучен на колонках: {num_imputer.feature_names_in_.tolist()}")
else:
    num_imputer = None # На случай, если нет числовых предикторов
    print("ПРЕДУПРЕЖДЕНИЕ: Нет числовых колонок для обучения Numerical Imputer (после исключения target-related).")

In [ ]:
'''from sklearn.impute import SimpleImputer

num_cols = train_data.select_dtypes(include=['int64', 'float64']).columns
imputer = SimpleImputer(strategy='mean')
train_data[num_cols] = imputer.fit_transform(train_data[num_cols])
train_data.head()'''

In [ ]:
train_data['director']

In [ ]:
nan_data = (train_data.isnull().mean() * 100).reset_index()
nan_data.columns = ["column_name", "percentage"]
nan_data.sort_values("percentage", ascending=False, inplace=True)
nan_data.head(24)

In [ ]:
genre_cols = ["genre_1", "genre_2", "genre_3", "genre_4"]
# число жанров
'''train_data["num_genres"] = train_data[genre_cols].notna().sum(axis=1)
genres = train_data[genre_cols].apply(lambda row: [g for g in row if pd.notna(g)], axis=1)
family_genres = {"Animation", "Family", "Adventure"}
train_data["is_family_friendly"] = genres.apply(lambda gl: int(bool(set(gl) & family_genres)))
'''

In [ ]:
# ==============================================================================
# >>> ОБНОВЛЕННЫЙ БЛОК: Групповая импутация и СОХРАНЕНИЕ КАРТ МОД <<<
# ==============================================================================

cols_for_grouped_imputation = ['cinematographer', 'composer', 'producer', 'writer'] # Переименовал для ясности

# Инициализируем словарь для карт мод, если еще не сделано
if 'grouped_imputation_maps_to_save' not in globals():
    grouped_imputation_maps_to_save = {}
    print("Инициализирован 'grouped_imputation_maps_to_save'")

print("\nГрупповая импутация (модой по 'director') и сохранение карт мод...")
if 'director' in train_data.columns:
    for col_to_impute in cols_for_grouped_imputation:
        if col_to_impute in train_data.columns:
            print(f"  Обработка колонки '{col_to_impute}'...")
            
            # 1. СОХРАНЯЕМ КАРТУ: director -> mode_of_col_to_impute
            # Эта карта будет содержать для каждого режиссера наиболее частый cinematographer/composer и т.д.
            # np.nan будет, если для режиссера все значения в col_to_impute были NaN или группа пуста
            director_to_mode_map = train_data.groupby('director')[col_to_impute].apply(
                lambda x: x.mode().iloc[0] if not x.mode().empty and not x.mode().isnull().all() else np.nan
            )
            grouped_imputation_maps_to_save[f"{col_to_impute}_director_mode_map"] = director_to_mode_map.to_dict()
            print(f"    Карта мод для '{col_to_impute}' по 'director' сохранена.")
            # print(f"    Пример карты для {col_to_impute}: {list(director_to_mode_map.items())[:2]}") # Для отладки

            # 2. ПРИМЕНЯЕМ ИМПУТАЦИЮ, используя созданную карту
            # Сначала получаем серию мод для каждого значения в 'director' текущей строки
            # Если режиссера нет в карте (например, новый режиссер в тестовых данных), map вернет NaN
            mapped_modes_for_imputation = train_data['director'].map(director_to_mode_map)
            
            # Заполняем пропуски в col_to_impute значениями из mapped_modes_for_imputation
            train_data[col_to_impute].fillna(mapped_modes_for_imputation, inplace=True)
            print(f"    Пропуски в '{col_to_impute}' заполнены с использованием сохраненной карты мод.")
        else:
            print(f"  Предупреждение: столбец '{col_to_impute}' не найден для групповой импутации.")
else:
    print("  Предупреждение: столбец 'director' не найден, групповая импутация не будет выполнена.")

print("\nСловарь grouped_imputation_maps_to_save (пример):")
for k in list(grouped_imputation_maps_to_save.keys())[:2]:
    print(f"  {k}: {list(grouped_imputation_maps_to_save[k].items())[:2]}...")

# ==============================================================================
# >>> КОНЕЦ ОБНОВЛЕННОГО БЛОКА <<<
# ==============================================================================

In [ ]:
columns_to_fill = ['genre_2', 'genre_3', 'genre_4', 'main_actor_4']
train_data[columns_to_fill] = train_data[columns_to_fill].fillna('Unknown')


In [ ]:
cat_cols = train_data.select_dtypes(include=['object']).columns

cat_imputer = SimpleImputer(strategy='most_frequent')
train_data[cat_cols] = cat_imputer.fit_transform(train_data[cat_cols])

In [ ]:
nan_data = (train_data.isnull().mean() * 100).reset_index()
nan_data.columns = ["column_name", "percentage"]
nan_data.sort_values("percentage", ascending=False, inplace=True)
nan_data.head(24)

In [ ]:
train_data['run_time']

In [ ]:
train_data.head()

In [ ]:
'''import re

def convert_runtime_to_minutes(value):
    match = re.match(r'(?:(\d+)\s*hr)?\s*(?:(\d+)\s*min)?', str(value))
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        return hours * 60 + minutes
    return None  # если формат не распознан

train_data['run_time'] = train_data['run_time'].apply(convert_runtime_to_minutes)
train_data.head()
'''

In [ ]:
# Вывод столбцов и колисество значений с ними
cat_cols = train_data.select_dtypes(include=['object', 'category']).columns.tolist()

cat_stats = pd.DataFrame({
    'Column': cat_cols,
    'Unique_Count': [train_data[col].nunique() for col in cat_cols],
    'Unique_Values': [train_data[col].unique() for col in cat_cols]
})

cat_stats = cat_stats.sort_values(by='Unique_Count', ascending=False)
print(cat_stats[['Column', 'Unique_Count']])

In [ ]:
columns_for_target_encoding = ['main_actor_4', 'main_actor_3', 'writer', 'main_actor_2', 'producer', 'director', 'main_actor_1', 'cinematographer', 'composer', 'distributor']
columns_for_one_hot = ['genre_2', 'genre_3', 'genre_4', 'genre_1', 'mpaa']

In [ ]:
train_data.columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe.fit(train_data[columns_for_one_hot])

# Трансформируем трейн и тест
train_encoded = ohe.transform(train_data[columns_for_one_hot])
#train_data = pd.get_dummies(train_data, columns=columns_for_one_hot)
train_encoded_df = pd.DataFrame(
    train_encoded,
    columns=ohe.get_feature_names_out(columns_for_one_hot),
    index=train_data.index
)
train_data = train_data.drop(columns_for_one_hot, axis=1).join(train_encoded_df)
train_data.columns

In [ ]:
import category_encoders as ce
encoder = ce.TargetEncoder(cols=columns_for_target_encoding)
train_data_encoded = encoder.fit_transform(train_data[columns_for_target_encoding], train_data['worldwide'])
train_data[columns_for_target_encoding] = train_data_encoded

In [ ]:
train_data.head()

In [ ]:
X = train_data.drop("worldwide", axis=1)
y = train_data["worldwide"]
#y = np.log1p(train_data["worldwide"])

cols_to_drop_for_X = ['worldwide']
if 'domestic' in train_data.columns and 'domestic' not in X.columns: # Если domestic не должен быть в X
    cols_to_drop_for_X.append('domestic')
if 'international' in train_data.columns and 'international' not in X.columns: # Если international не должен быть в X
    cols_to_drop_for_X.append('international')

# Убедимся, что удаляем только существующие
actual_cols_to_drop_for_X = [col for col in cols_to_drop_for_X if col in train_data.columns]

X = train_data.drop(columns=actual_cols_to_drop_for_X)
y = train_data["worldwide"]
print(f"Колонки в X (финальные предикторы, первые 10): {X.columns.tolist()[:10]}")

In [ ]:
X.columns

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,MaxAbsScaler, QuantileTransformer, PowerTransformer
scaler = StandardScaler()
#scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

'''model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

print(f"R²: {r2_score(y_val, y_pred):.4f}") #R²: 0.9981
print(f"MSE: {mean_squared_error(y_val, y_pred):.2f}")#MSE: 73899943426108.95'''



'''rf = RandomForestRegressor(random_state=42)

# Сетка гиперпараметров
param_grid = {
    'n_estimators': [100, 1000],           # Кол-во деревьев
    'max_depth': [10, 50, None],          # Максимальная глубина
    'min_samples_split': [2, 10],          # Мин. объектов для разделения
    'min_samples_leaf': [1, 5],           # Мин. объектов в листе
    'max_features': ['auto', 'sqrt']      # Кол-во признаков при делении
}

# GridSearch с кросс-валидацией
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=3,
                           scoring='r2',
                           n_jobs=-1,
                           verbose=1)

# Обучение модели
grid_search.fit(X_train, y_train)

# Лучшая модель и параметры
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Предсказания и метрики
y_pred = best_model.predict(X_val)
r2 = r2_score(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print(f"Test R² Score: {r2:.4f}") # Test R² Score: 0.9801
print(f"Test MSE: {mse:.2f}") #Test MSE: 756709767970853.50'''



'''from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=8, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

print(f"R²: {r2_score(y_val, y_pred):.4f}")#R²: 0.9984
print(f"MSE: {mean_squared_error(y_val, y_pred):.2f}")#MSE: 61650183675109.39'''



'''from lightgbm import LGBMRegressor

model = LGBMRegressor(n_estimators=1000, learning_rate=0.21, max_depth=-1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

print(f"R²: {r2_score(y_val, y_pred):.4f}") # R²: 0.9953
print(f"MSE: {mean_squared_error(y_val, y_pred):.2f}") #MSE: 180661829545789.47'''



'''from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

print(f"R²: {r2_score(y_val, y_pred):.4f}") # MSE: 145843899870827.59
print(f"MSE: {mean_squared_error(y_val, y_pred):.2f}") # R²: 0.9962'''



param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth':    [4, 6, 8],
    'learning_rate':[0.01, 0.05, 0.1],
    'subsample':    [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha':    [0, 0.5, 1],
    'reg_lambda':   [1, 2, 5]
}

# 4) Создаём XGB с фиксированным seed и детерминированным методом
base_model = XGBRegressor(
    random_state=42,
    tree_method='hist',            # более детерминированный
    enable_categorical=False,
    n_jobs=1
)

# 5) RandomizedSearchCV с фиксированным seed
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=1,              # лучше 1, чтобы исключить недетерминированность
    random_state=42
)

# 6) Обучаем
random_search.fit(X_train, y_train)

# 7) Оцениваем
best = random_search.best_estimator_
y_pred = best.predict(X_val)
print(f"R²:  {r2_score(y_val, y_pred):.4f}")
print(f"MSE: {mean_squared_error(y_val, y_pred):.2f}")
#Best Params: {'subsample': 0.8, 'reg_lambda': 5, 'reg_alpha': 1, 'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
#Best R² Score on Test: 0.9990
#Best MSE on Test: 37542782216082.52

#Best R² Score on Test: 0.9990
#Best MSE on Test: 36496695202244.70

#Best R² Score on Test: 0.9991
#Best MSE on Test: 32663862493933.65


#R²:  0.9993
#MSE: 28215564467164.59


'''param_dist = {
    'iterations': [200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'random_strength': [0.5, 1, 2]
}

# Создаём модель
cat = CatBoostRegressor(verbose=0, random_seed=42)

# RandomizedSearch
random_search = RandomizedSearchCV(
    estimator=cat,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Обучение
random_search.fit(X_train, y_train)

# Лучшая модель
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Предсказания и метрики
y_pred = best_model.predict(X_val)
r2 = r2_score(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print(f"Best R²: {r2:.4f}")
print(f"Best MSE: {mse:.2f}")
#Best Parameters: {'subsample': 0.8, 'random_strength': 0.5, 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 300, 'depth': 6}
#Best R²: 0.9978
#Best MSE: 83318573388615.64 '''




'''param_dist = {
    'n_estimators': [200, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [-1, 4, 6, 8, 10],
    'num_leaves': [31, 50, 100, 150],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1]
}

# Модель
lgb = LGBMRegressor(random_state=42)

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=lgb,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Обучение
random_search.fit(X_train, y_train)

# Лучшая модель и параметры
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Предсказание и метрики
y_pred = best_model.predict(X_val)
r2 = r2_score(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print(f"Best R²: {r2:.4f}")
print(f"Best MSE: {mse:.2f}")
#Best Parameters: {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.1, 'num_leaves': 31, 'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
#Best R²: 0.9959
#Best MSE: 157899903576786.03'''


In [ ]:
'''param_dist = {
    'n_estimators': [200, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [-1, 4, 6, 8, 10],
    'num_leaves': [31, 50, 100, 150],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1]
}

# Модель
lgb = LGBMRegressor(random_state=42)

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=lgb,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Обучение
random_search.fit(X_train, y_train)

# Лучшая модель и параметры
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Предсказание и метрики
y_pred = best_model.predict(X_val)# Обратное преобразование: экспоненцируем предсказания
y_pred_original = np.expm1(y_pred)  # Возвращаем обратно в исходный масштаб

# Метрики
r2 = r2_score(np.expm1(y_val), y_pred_original)  # Обратно логарифмируем y_val для расчета метрик
mse = mean_squared_error(np.expm1(y_val), y_pred_original)
print(f"Best R²: {r2:.4f}")
print(f"Best MSE: {mse:.2f}")
#Best R²: 0.9981
#Best MSE: 73636379429094.73'''




'''param_dist = {
    'iterations': [200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'random_strength': [0.5, 1, 2]
}

# Создаём модель
cat = CatBoostRegressor(verbose=0, random_seed=42)

# RandomizedSearch
random_search = RandomizedSearchCV(
    estimator=cat,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Обучение
random_search.fit(X_train, y_train)

# Лучшая модель
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Предсказания и метрики
y_pred = best_model.predict(X_val)
y_pred_original = np.expm1(y_pred)  # Возвращаем обратно в исходный масштаб

# Метрики
r2 = r2_score(np.expm1(y_val), y_pred_original)  # Обратно логарифмируем y_val для расчета метрик
mse = mean_squared_error(np.expm1(y_val), y_pred_original)

print(f"Best R²: {r2:.4f}")
print(f"Best MSE: {mse:.2f}")
#Best R²: 0.9934
#Best MSE: 252026444906922.81'''



'''param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 2, 5]
}

random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print("Best Params:", random_search.best_params_)
from sklearn.metrics import r2_score, mean_squared_error

# Лучшая модель из RandomizedSearchCV
best_model = random_search.best_estimator_

# Предсказания на тестовой выборке
y_pred = best_model.predict(X_val)
y_pred_original = np.expm1(y_pred)  # Возвращаем обратно в исходный масштаб

# Метрики
r2 = r2_score(np.expm1(y_val), y_pred_original)  # Обратно логарифмируем y_val для расчета метрик
mse = mean_squared_error(np.expm1(y_val), y_pred_original)

print(f"Best R² Score on Test: {r2:.4f}")
print(f"Best MSE on Test: {mse:.2f}")
#Best R² Score on Test: 0.9975
#Best MSE on Test: 95385734348795.59'''





'''rf = RandomForestRegressor(random_state=42)

# Сетка гиперпараметров
param_grid = {
    'n_estimators': [100, 1000],           # Кол-во деревьев
    'max_depth': [10, 50, None],          # Максимальная глубина
    'min_samples_split': [2, 10],          # Мин. объектов для разделения
    'min_samples_leaf': [1, 5],           # Мин. объектов в листе
    'max_features': ['auto', 'sqrt']      # Кол-во признаков при делении
}

# GridSearch с кросс-валидацией
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=3,
                           scoring='r2',
                           n_jobs=-1,
                           verbose=1)

# Обучение модели
grid_search.fit(X_train, y_train)

# Лучшая модель и параметры
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Предсказания и метрики
y_pred = best_model.predict(X_val)
y_pred_original = np.expm1(y_pred)  # Возвращаем обратно в исходный масштаб

# Метрики
r2 = r2_score(np.expm1(y_val), y_pred_original)  # Обратно логарифмируем y_val для расчета метрик
mse = mean_squared_error(np.expm1(y_val), y_pred_original)

print(f"Test R² Score: {r2:.4f}")
print(f"Test MSE: {mse:.2f}") 
#Test R² Score: 0.9648
#Test MSE: 1341694408103681.00'''




TEST DATA

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
test_movie_title = test_data['movie_title']
columns_to_drop = ['movie_id', 'movie_title', 'link']
#columns_to_drop = ['movie_id', 'link']
test_data = test_data.drop(columns=columns_to_drop)

In [ ]:
for col in ["director", "writer", "producer", "composer", "cinematographer"]:
    counts = train_data[col].value_counts()
    test_data[f"{col}_experience"] = test_data[col].map(counts).fillna(0)
for i in range(1, 5):
    col = f"main_actor_{i}"
    counts = train_data[col].value_counts()
    test_data[f"{col}_experience"] = test_data[col].map(counts).fillna(0)

test_data["cast_popularity"] = sum(test_data[f"main_actor_{i}_experience"] for i in range(1, 5))
test_data.head()

In [ ]:
def convert_runtime_to_minutes(value):
    match = re.match(r'(?:(\d+)\s*hr)?\s*(?:(\d+)\s*min)?', str(value))
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        return hours * 60 + minutes
    return None  # если формат не распознан

test_data['run_time'] = test_data['run_time'].apply(convert_runtime_to_minutes)

In [ ]:
test_num_cols = test_data.select_dtypes(include=['int64', 'float64']).columns
print(test_num_cols)
test_data[test_num_cols] = num_imputer.transform(test_data[test_num_cols])

In [ ]:
cols_to_fill = ['cinematographer', 'composer', 'producer', 'writer']

for col in cols_to_fill:
    mode_values = train_data.groupby('director')[col].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    test_data[col] = test_data.apply(lambda row: mode_values.get(row['director'], np.nan) if pd.isnull(row[col]) else row[col], axis=1)


In [ ]:
columns_to_fill = ['genre_2', 'genre_3', 'genre_4', 'main_actor_4']
test_data[columns_to_fill] = test_data[columns_to_fill].fillna('Unknown')

cat_cols = test_data.select_dtypes(include=['object']).columns

test_data[cat_cols] = cat_imputer.fit_transform(test_data[cat_cols])

In [ ]:
nan_data = (test_data.isnull().mean() * 100).reset_index()
nan_data.columns = ["column_name", "percentage"]
nan_data.sort_values("percentage", ascending=False, inplace=True)
nan_data.head(24)

In [ ]:
test_encoded = ohe.transform(test_data[columns_for_one_hot])
test_encoded_df = pd.DataFrame(
    test_encoded,
    columns=ohe.get_feature_names_out(columns_for_one_hot),
    index=test_data.index
)

test_data = test_data.drop(columns_for_one_hot, axis=1).join(test_encoded_df)
test_data = test_data.reindex(columns=train_data.columns, fill_value=0)
test_data.columns


In [ ]:
test_data_encoded = encoder.fit_transform(test_data[columns_for_target_encoding], test_data['worldwide'])
test_data[columns_for_target_encoding] = test_data_encoded


In [ ]:
X_test =test_data.drop("worldwide", axis=1)
y_test = test_data["worldwide"]

In [ ]:
train_data.columns

In [ ]:
test_data.columns

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
y_pred = best.predict(X_test)
print(f"R²:  {r2_score(y_test, y_pred):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")

In [ ]:
y_pred

In [ ]:
# ==============================================================================
# >>> ОБНОВЛЕННЫЙ БЛОК СОХРАНЕНИЯ АРТЕФАКТОВ <<<
# Использует experience_maps_to_save и grouped_imputation_maps_to_save,
# которые должны быть определены и заполнены ранее в ноутбуке.
# ==============================================================================
import joblib
import json
import os
import pandas as pd # Убедимся, что pandas импортирован для pd.isna
import numpy as np  # Убедимся, что numpy импортирован для np.nan

# --- 0. Определяем директорию для сохранения ---
MODEL_SAVE_DIR = "saved_model" 
if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)
print(f"Артефакты будут сохранены в директорию: {MODEL_SAVE_DIR}")

# --- 1. Сохранение обученной модели ---
# Ваш 'best' - это XGBRegressor (обертка sklearn)
# Если вы хотите сохранить его как XGBRegressor, используйте joblib
model_filename_joblib = os.path.join(MODEL_SAVE_DIR, "movie_box_office_model.joblib")
joblib.dump(best, model_filename_joblib)
print(f"Модель (XGBRegressor) сохранена как: {model_filename_joblib}")

# Если вы хотите сохранить XGBoost Booster в JSON формате (как в вашем tree для predictor.py):
# Убедитесь, что 'best' это действительно XGBoost модель, а не, например, Pipeline
if hasattr(best, 'save_model'): # Проверка, есть ли у объекта метод save_model (как у XGBoost Booster/Regressor)
    model_filename_json = os.path.join(MODEL_SAVE_DIR, "movie_box_office_model.json")
    try:
        # Если 'best' это scikit-learn обертка XGBRegressor, то get_booster() вернет "сырой" Booster
        booster_to_save = best.get_booster() if hasattr(best, 'get_booster') else best
        booster_to_save.save_model(model_filename_json)
        print(f"Модель XGBoost (Booster) также сохранена в JSON формате: {model_filename_json}")
    except Exception as e_save_json:
        print(f"Не удалось сохранить модель в JSON формате: {e_save_json}. Используйте .joblib версию.")
else:
    print("Модель 'best' не имеет метода save_model, JSON версия не сохранена.")


# --- 2. Сохранение трансформеров ---
joblib.dump(num_imputer, os.path.join(MODEL_SAVE_DIR, "numerical_imputer.joblib"))
print("Numerical Imputer сохранен.")

joblib.dump(cat_imputer, os.path.join(MODEL_SAVE_DIR, "categorical_imputer.joblib"))
print("Categorical Imputer сохранен.")

joblib.dump(ohe, os.path.join(MODEL_SAVE_DIR, "onehot_encoder.joblib")) # ohe - ваш sklearn.preprocessing.OneHotEncoder
print("OneHotEncoder сохранен.")

joblib.dump(encoder, os.path.join(MODEL_SAVE_DIR, "target_encoder.joblib")) # encoder - ваш ce.TargetEncoder
print("Target Encoder сохранен.")

joblib.dump(scaler, os.path.join(MODEL_SAVE_DIR, "scaler.joblib")) # scaler - ваш StandardScaler
print("Scaler сохранен.")


# --- 3. Сохранение информации о колонках и карт преобразований ---

# Списки колонок, определенные в вашем ноутбуке:
# columns_to_drop - должен быть определен ранее, например: ['movie_id', 'movie_title', 'link']
# columns_for_one_hot - должен быть определен ранее, например: ['genre_2', 'genre_3', 'genre_4', 'genre_1', 'mpaa']
# columns_for_target_encoding - должен быть определен ранее

# Финальные признаки для модели (X.columns.tolist() ПОСЛЕ OHE и Target Encoding, ДО scaling)
# Убедитесь, что DataFrame 'X' существует на этом этапе и содержит правильные колонки
if 'X' not in globals() or not isinstance(X, pd.DataFrame):
    raise NameError("DataFrame 'X' с финальными признаками не определен перед сохранением column_info.")
final_model_features_list = X.columns.tolist()

# Колонки, на которых обучался numerical_imputer
# Атрибут feature_names_in_ хранит имена колонок, на которых был сделан fit
if not hasattr(num_imputer, 'feature_names_in_'):
    raise AttributeError("Атрибут 'feature_names_in_' отсутствует у num_imputer. Убедитесь, что импьютер был обучен.")
numerical_imputer_cols_fitted_on = num_imputer.feature_names_in_.tolist()
# ВАЖНО: Если 'domestic', 'international', 'worldwide' НЕ являются предикторами,
# они не должны быть в этом списке. Это значит, что num_imputer должен был обучаться
# на DataFrame, из которого эти колонки уже удалены (если они не предикторы).
# Если они попали в fit, predictor.py должен будет их ожидать.
# Рекомендуется обучать num_imputer только на тех числовых колонках, которые являются предикторами и могут иметь пропуски.


# Колонки, на которых обучался categorical_imputer
if not hasattr(cat_imputer, 'feature_names_in_'):
    raise AttributeError("Атрибут 'feature_names_in_' отсутствует у cat_imputer.")
categorical_imputer_cols_fitted_on = cat_imputer.feature_names_in_.tolist()

# Колонки, на которые подавался OneHotEncoder (исходные)
# columns_for_one_hot - это ваш список исходных колонок для OHE
ohe_input_columns_list = columns_for_one_hot

# Колонки для TargetEncoder (исходные)
# columns_for_target_encoding - ваш список
target_encoding_columns_list = columns_for_target_encoding


# Используем experience_maps_to_save и grouped_imputation_maps_to_save,
# которые вы создали и заполнили ранее в ноутбуке.
if 'experience_maps_to_save' not in globals() or 'grouped_imputation_maps_to_save' not in globals():
    print("ПРЕДУПРЕЖДЕНИЕ: 'experience_maps_to_save' или 'grouped_imputation_maps_to_save' не найдены.")
    print("              Карты опыта и/или групповой импутации могут быть не сохранены или сохранены пустыми.")
    # Создаем пустые, чтобы код не упал, но это будет некорректно для predictor.py
    experience_maps_to_save = experience_maps_to_save if 'experience_maps_to_save' in globals() else {}
    grouped_imputation_maps_to_save = grouped_imputation_maps_to_save if 'grouped_imputation_maps_to_save' in globals() else {}


column_info_data = {
    "features": final_model_features_list, # Финальные признаки для модели
    "target": "worldwide",                 # Имя целевой переменной
    
    "initial_columns_to_drop": columns_to_drop, # Начальные колонки для удаления
    "ohe_input_columns": ohe_input_columns_list, # Исходные колонки для OneHotEncoder
    "target_encoding_columns": target_encoding_columns_list, # Исходные колонки для TargetEncoder

    "numerical_imputer_features_in": numerical_imputer_cols_fitted_on,
    "categorical_imputer_features_in": categorical_imputer_cols_fitted_on,

    "experience_maps": experience_maps_to_save, 
    "grouped_imputation_maps": grouped_imputation_maps_to_save,
    
    # Списки колонок, которые могут помочь predictor.py понять логику вашего ноутбука
    "personnel_cols_for_experience": ["director", "writer", "producer", "composer", "cinematographer"],
    "actor_cols_for_experience_prefix": "main_actor_",
    # Колонки, которые импьютировались модой по 'director'
    "cols_for_grouped_imputation_source_director": ['cinematographer', 'composer', 'producer', 'writer'], # Из вашего кода
    # Колонки, которые заполнялись 'Unknown'
    "cols_to_fill_unknown_specific": ['genre_2', 'genre_3', 'genre_4', 'main_actor_4'] # Из вашего кода
}

# Очистка np.nan для JSON сериализации в картах
def clean_map_for_json(data_map):
    if not isinstance(data_map, dict): return {} # Если не словарь, возвращаем пустой
    cleaned = {}
    for map_key, inner_dict in data_map.items():
        if isinstance(inner_dict, dict):
             cleaned[map_key] = {str(k): (None if pd.isna(v) else v) for k, v in inner_dict.items()}
        else: # Если значение в data_map не словарь (маловероятно для карт, но для безопасности)
            cleaned[map_key] = None if pd.isna(inner_dict) else inner_dict 
    return cleaned

# Применяем очистку только если карты не пустые
if column_info_data["experience_maps"]:
    column_info_data["experience_maps"] = clean_map_for_json(column_info_data["experience_maps"])
if column_info_data["grouped_imputation_maps"]:
    column_info_data["grouped_imputation_maps"] = clean_map_for_json(column_info_data["grouped_imputation_maps"])


column_info_filename = os.path.join(MODEL_SAVE_DIR, "column_info.json")
with open(column_info_filename, "w", encoding='utf-8') as f:
    json.dump(column_info_data, f, indent=2, ensure_ascii=False)
print(f"Информация о колонках сохранена как: {column_info_filename}")

# --- 4. (Опционально) Сохранение all_data.csv, если generate_options.py его оттуда читает ---
# import shutil
# source_all_data = "all_data.csv" # Укажите правильный путь к вашему all_data.csv
# if os.path.exists(source_all_data):
#    dest_all_data = os.path.join(MODEL_SAVE_DIR, "all_data.csv")
#    shutil.copyfile(source_all_data, dest_all_data)
#    print(f"Файл {source_all_data} скопирован в {dest_all_data}")
# else:
#    print(f"ПРЕДУПРЕЖДЕНИЕ: Исходный файл {source_all_data} не найден, не скопирован.")

print(">>> СОХРАНЕНИЕ АРТЕФАКТОВ ЗАВЕРШЕНО <<<")
# ==============================================================================